In [2]:
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Extract Dataset

In [3]:
base_dir = 'PlantVillage'

# Membaca Direktori
print("Contents of Base Directory : ")
print(os.listdir(base_dir))

# Membaca Sub-Direktori
print("\nContents of Base Directory : ")
print(os.listdir(f'{base_dir}/Training'))

# Membaca Sub-Direktori
print("\nContents of Base Directory : ")
print(os.listdir(f'{base_dir}/Validation'))


Contents of Base Directory : 
['Training', 'Validation']

Contents of Base Directory : 
['Potato Early Blight', 'Potato Healthy', 'Potato Late Blight']

Contents of Base Directory : 
['Potato Early Blight', 'Potato Healthy', 'Potato Late Blight']


## Akses Dataset

In [4]:
data_list = ['Potato Early Blight','Potato Healthy','Potato Late Blight']

for data in data_list:
    print(f'Total Training {data} Images : ', len(os.listdir(f'{base_dir}/Training/{data}')))
    
print(" ------------------------------------------------ ")

for data in data_list:
    print(f'Total Validation {data} Images : ', len(os.listdir(f'{base_dir}/Validation/{data}')))

Total Training Potato Early Blight Images :  800
Total Training Potato Healthy Images :  121
Total Training Potato Late Blight Images :  800
 ------------------------------------------------ 
Total Validation Potato Early Blight Images :  200
Total Validation Potato Healthy Images :  31
Total Validation Potato Late Blight Images :  200


## Resize Dataset 

In [5]:
from PIL import Image
import os

def resize_images(input_dir, output_dir, target_size):
    for dataset_type in ['Training', 'Validation']:
        input_dataset_dir = os.path.join(input_dir, dataset_type)
        output_dataset_dir = os.path.join(output_dir, dataset_type)

        if not os.path.exists(output_dataset_dir):
            os.makedirs(output_dataset_dir)

        for class_name in os.listdir(input_dataset_dir):
            input_class_dir = os.path.join(input_dataset_dir, class_name)
            output_class_dir = os.path.join(output_dataset_dir, class_name)

            if not os.path.exists(output_class_dir):
                os.makedirs(output_class_dir)

            for filename in os.listdir(input_class_dir):
                input_img_path = os.path.join(input_class_dir, filename)
                output_img_path = os.path.join(output_class_dir, filename)

                img = Image.open(input_img_path)
                resized_img = img.resize(target_size)
                resized_img.save(output_img_path)

input_dir = './PlantVillage/'
output_dir = './resized_PlantVillage'

# Adjust target size as needed
target_size = (224, 224)

resize_images(input_dir, output_dir, target_size)

print("Resize 224 x 224 piksel Telah Selesai Dilakukan")

Resize 224 x 224 piksel Telah Selesai Dilakukan


## Akses Data yang telah di resize 224 x 224 pixel

In [6]:
data_list = ['Potato Early Blight','Potato Healthy','Potato Late Blight']

for data in data_list:
    print(f'Total Training {data} Resize Images : ', len(os.listdir(f'{output_dir}/Training/{data}')))
    
print("======================================================")

for data in data_list:
    print(f'Total Validation {data} Resize Images : ', len(os.listdir(f'{output_dir}/Validation/{data}')))

Total Training Potato Early Blight Resize Images :  800
Total Training Potato Healthy Resize Images :  121
Total Training Potato Late Blight Resize Images :  800
Total Validation Potato Early Blight Resize Images :  200
Total Validation Potato Healthy Resize Images :  31
Total Validation Potato Late Blight Resize Images :  200


In [7]:
# Membuat Variabel baru untuk direktori resized
TRAIN_RESIZED_DIR = os.path.join(output_dir, 'Training')
VALIDATION_RESIZED_DIR = os.path.join(output_dir, 'Validation')

# Directory with training cat/dog pictures
train_early_resized_dir = os.path.join(TRAIN_RESIZED_DIR, 'Potato Early Blight')
train_healthy_resized_dir = os.path.join(TRAIN_RESIZED_DIR, 'Potato Healthy')
train_late_resized_dir = os.path.join(TRAIN_RESIZED_DIR, 'Potato Late Blight')

# Directory with validation cat/dog pictures
validation_early_resized_dir = os.path.join(VALIDATION_RESIZED_DIR, 'Potato Early Blight')
validation_healthy_resized_dir = os.path.join(VALIDATION_RESIZED_DIR, 'Potato Healthy')
validation_late_resized_dir = os.path.join(VALIDATION_RESIZED_DIR, 'Potato Late Blight')

## Data Augmentation

In [8]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    horizontal_flip =True,
    zoom_range = 0.2,
    shear_range = 0.2,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_RESIZED_DIR,
    target_size = (224, 224),
    batch_size= 32,
    shuffle = True,
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_RESIZED_DIR,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1721 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


## Create Base Model MobileNetV2

In [9]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    )

base_model.trainable = False


In [15]:
total_params = base_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in base_model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

There are 2,257,984 total parameters in this model.
There are 0 trainable parameters in this model.


In [16]:
final_model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

final_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 128)               163968    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 3)                 387       
                                                                 
Total params: 2422339 (9.24 MB)
Trainable params: 1643

In [17]:
final_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model_fit = final_model.fit(
    train_generator,
    validation_data  = validation_generator,
    epochs = 5,
    verbose = 2
)

Epoch 1/5
54/54 - 29s - loss: 0.3217 - accuracy: 0.8786 - val_loss: 0.1433 - val_accuracy: 0.9443 - 29s/epoch - 538ms/step
Epoch 2/5
54/54 - 18s - loss: 0.1059 - accuracy: 0.9611 - val_loss: 0.1181 - val_accuracy: 0.9559 - 18s/epoch - 325ms/step
Epoch 3/5
54/54 - 17s - loss: 0.0973 - accuracy: 0.9582 - val_loss: 0.1029 - val_accuracy: 0.9466 - 17s/epoch - 322ms/step
Epoch 4/5
54/54 - 17s - loss: 0.0903 - accuracy: 0.9646 - val_loss: 0.0976 - val_accuracy: 0.9652 - 17s/epoch - 323ms/step
Epoch 5/5
54/54 - 19s - loss: 0.0758 - accuracy: 0.9709 - val_loss: 0.0917 - val_accuracy: 0.9606 - 19s/epoch - 345ms/step
